In [1]:
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
import numpy as np
np.set_printoptions(precision=5, linewidth=150)
import spax
from datetime import datetime
from sklearn.decomposition import PCA

# Principle Component Analysis (PCA)
Let X be $n \times m$ matrix, where $n$ represents dimensionality of the data and $m$ number of samples.
To prepare data for the PCA, it should be centered `X -= X.mean(axis = 1)`, and possibly whitened `X /= X.std(axis = 1)`. Whitening is done to scale dimensions to unit variance. This is specifically useful if data dimensions are in different units or represent different observables. PCA then consists of finding a rotation matrix which makes covariance of the data diagonal. Specifically, PCA of order $N$ does this rotation for $N$ largest eigenvalues of the covariance matrix.

## Singular Value Decomposition (SVD)
Every non-quadratic matrix can be decomposed as $$X = U \, S \, V^T \, ,$$ where $U$ is $n \times m$ matrix with orthonormal columns ($U^T \, U = I$), $V$ is $m \times m$ orthonormal matrix ($V^T \, V = I$) and $S$ a diagonal $m\times m$ matrix of singular values. From such decomposition one can write
\begin{align}
X \, X^T &= U \, S^2 \, U^T  \, , \\
X^T X &= V \, S^2 \, V^T \, .
\end{align}
Both $X \, X^T$ and $X^T X$ have the same eigenvalues, with larger of the two having the rest equal to $0$.

## Case #1: $n \le m$
Covariance of the data can be written as $$ C = \frac{1}{m-1} X \, X^T = U \, \frac{S^2}{m-1} \, U^T \, .$$ Therefore, by solving an eigenvalue problem for $C$, we can find $U$. By picking $N$ eigenvectors in the directions of the largest eigenvalues, we construct $\widetilde{U}$ used for PCA.

For some matrix $X_0$ of size $n \times m_0$, PCA is simply $$ Y_0 = \widetilde{U}^T X_0$$.

In [2]:
N_dim, N_samples = (16, 10**5)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2, centering_data = "GPU") # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(N_samples, batch_size = N_dim // 2)
print(np.std(pca.transform(data, batch_size = N_dim // 2), axis = 1)**2) # should be [8, 7, 6, 5, 4]
print(np.std(pca.transform(sampled_data, batch_size = N_dim // 2), axis = 1)**2) #should be the same
print(pca.λ ** 2) # should be the same
print(np.round(pca.U.T, 1)) # should be a +-unit matrix on last 5 dimensions

DURATION: 0:00:03.745769
[16.00737 15.09035 13.97557 12.95597 11.96924]
[15.9131  15.14852 13.93848 12.92286 11.9518 ]
[16.00817 15.0917  13.97685 12.95663 11.97026]
[[-0.   0.  -0.   0.   0.  -0.  -0.   0.   0.  -0.  -0.   0.  -0.  -0.   0.   1. ]
 [ 0.   0.  -0.  -0.   0.   0.  -0.   0.  -0.   0.  -0.   0.   0.  -0.   1.  -0. ]
 [-0.   0.  -0.   0.   0.  -0.   0.  -0.  -0.  -0.   0.   0.  -0.   1.   0.   0. ]
 [-0.   0.  -0.   0.  -0.  -0.   0.   0.   0.   0.  -0.  -0.1  1.   0.  -0.   0. ]
 [-0.  -0.   0.  -0.  -0.   0.   0.   0.  -0.   0.   0.   1.   0.1 -0.  -0.  -0. ]]


In [3]:
#testing the result with scikit-learn
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(data.T), axis = 0)**2)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)**2)
print(pca_sk.singular_values_**2 / (N_samples - 1))
print(np.round(pca_sk.components_, 1))

DURATION: 0:00:00.250611
[16.00795 15.09157 13.97652 12.95648 11.97058]
[15.91202 15.1465  13.93696 12.9219  11.94907]
[16.00811 15.09173 13.97666 12.95661 11.9707 ]
[[ 0.   0.  -0.   0.   0.  -0.   0.   0.   0.  -0.  -0.   0.  -0.  -0.   0.   1. ]
 [-0.  -0.   0.   0.   0.   0.  -0.  -0.  -0.   0.  -0.   0.   0.  -0.   1.  -0. ]
 [-0.  -0.  -0.   0.  -0.  -0.   0.  -0.  -0.  -0.   0.   0.  -0.   1.   0.   0. ]
 [ 0.  -0.  -0.   0.   0.  -0.  -0.  -0.  -0.  -0.   0.   0.1 -1.  -0.   0.  -0. ]
 [-0.  -0.  -0.   0.  -0.  -0.   0.  -0.   0.  -0.  -0.  -1.  -0.1  0.   0.   0. ]]


## Case #2 $n \ge m$
In this case, it is better to write: $$D = \frac{1}{n} X^T \, X = V \, \frac{S^2}{n} \, V^T \, .$$
Solving eigenvector problem for $D$ gives us $V$ and $S$. Then, rotation matrix can be computed as $$ U = X \, V \, S^{-1} \, .$$ The rest is the same as in previous case.

In [4]:
N_dim, N_samples = (10**5, 16)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2, centering_data = "GPU") # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(10**4, batch_size = N_dim // 2)
print(np.std(pca.transform(data, batch_size = N_dim // 2), axis = 1)**2)
print(np.std(pca.transform(sampled_data, batch_size = N_dim // 2), axis = 1)**2)
print(pca.λ ** 2)
print(pca.U.T)

DURATION: 0:00:01.656207
[3.19257e+08 3.18094e+08 3.17750e+08 3.16736e+08 3.15445e+08]
[3.37049e+08 3.40064e+08 3.36459e+08 3.38866e+08 3.35253e+08]
[3.40541e+08 3.39301e+08 3.38934e+08 3.37852e+08 3.36476e+08]
[[ 5.44903e-06 -7.18126e-06 -2.32861e-05 ...  1.17157e-03  3.70124e-03 -3.71298e-03]
 [-6.53237e-06  6.45604e-06  7.23129e-06 ...  4.39756e-03  1.25853e-03  9.96697e-04]
 [ 1.45524e-05 -7.21100e-06 -2.33295e-05 ...  7.44863e-03  7.44766e-03 -4.16233e-03]
 [ 7.94391e-06  4.48932e-05 -8.15404e-06 ... -4.68911e-03  2.85591e-03 -2.90643e-03]
 [-6.64980e-06 -1.88985e-07 -1.97219e-05 ...  4.93046e-03  4.89242e-03 -8.78050e-04]]


In [5]:
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(data.T), axis = 0)**2)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)**2)
print(pca_sk.singular_values_**2 / (N_samples - 1))
print(pca_sk.components_)

DURATION: 0:00:00.176588
[3.19257e+08 3.18094e+08 3.17750e+08 3.16735e+08 3.15445e+08]
[3.37049e+08 3.40065e+08 3.36460e+08 3.38864e+08 3.35254e+08]
[3.40541e+08 3.39300e+08 3.38933e+08 3.37851e+08 3.36475e+08]
[[ 5.44923e-06 -7.18217e-06 -2.32867e-05 ...  1.17190e-03  3.70141e-03 -3.71311e-03]
 [ 6.52941e-06 -6.45714e-06 -7.22693e-06 ... -4.39888e-03 -1.26010e-03 -9.96016e-04]
 [ 1.45554e-05 -7.20253e-06 -2.33319e-05 ...  7.44681e-03  7.44791e-03 -4.16299e-03]
 [-7.94253e-06 -4.48929e-05  8.14821e-06 ...  4.69115e-03 -2.85369e-03  2.90554e-03]
 [ 6.64819e-06  1.85479e-07  1.97229e-05 ... -4.93024e-03 -4.89275e-03  8.78251e-04]]


## Test save + load

In [6]:
pca.save("test.hdf5")
pca_new = spax.PCA_m(5, devices = jax.devices("gpu"))
pca_new.load("test.hdf5")

x0 = np.arange(100000)[:, np.newaxis]
print(pca.transform(x0).flatten())
print(pca_new.transform(x0).flatten())
print(pca_sk.transform(x0.T).flatten())

[ 42476.547 138688.97  -25199.688  -5766.743  30936.84 ]
[ 42476.547 138688.94  -25199.688  -5766.748  30936.838]
[  42477.39484 -138682.33761  -25228.97732    5778.40298  -30929.59941]
